# *Lab 3 Manual: Basics of Neural Networks with Tensorflow*

# **Libraries import and Data Cleaning**

**pandas:** the first library we learned, used for loading and handling datasets.

**numpy:** helps with math operations and working with arrays.

**tensorflow / keras:** used to build and train our neural network model.

**train_test_split:** splits the data into training and testing sets.

**StandardScaler:** prepares the data by scaling the features for better model performance.

In [ ]:
#arsenal

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import Precision, Recall


Next step is to load and prepare the data. You can refer to Lab Manual 1 for more information about Data Cleaning.

In [ ]:
df = pd.read_csv('/kaggle/input/fantasy-football/master_team_list.csv')
# Basic cleaning
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
# Drop irrelevant columns
df = df.drop(columns=['Cabin', 'Name', 'Ticket'])


Let's take a look at the data after applying the preprocessing techniques

In [ ]:
df.head(10)

# **Defining Features and Target** 
Before training a neural network model, we need to extract the features and the target (ground truth) from the dataset, depending on the task we want the model to learn.

**Features (X):** We select the columns Pclass, Sex, Age, Fare, SibSp, and Parch from the dataset. These variables will be used as inputs to the model.

**Target (y):** We choose the Survived column as the target variable, which represents the outcome we want the model to predict.

In [ ]:
# Feature and target
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']]
y = df['Survived']

In [ ]:
# Show the first 5 rows of features
print("Features (X):")
print(X.head())

# Show the first 5 rows of target values
print("\nTarget (y):")
print(y.head())

#Show the dimensions of both the Features and Targets dataframes
print("X and y dimensions:",X.shape,y.shape)

# **Features Scaling**

We usually scale features before training machine learning models because raw features can have very different ranges and units, which may cause problems.

For example, in the Titanic dataset:

* Age ranges from about 0.1–80 years

* Fare ranges from about 20–500 dollars

Without scaling, algorithms that rely on distances or weights may treat Fare as more important than Age simply because the numbers are larger.

Standardization solves this by transforming each feature. We don't need to do that with the target labels, only the features.

In [ ]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Print first 5 rows of scaled features
print("Scaled Features (first 5 rows):")
print(X_scaled[:5])

print("Shape of 1 row:")
print(X_scaled[0].shape)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
#Explore the size and dimension of the train and test data after splitting:
print("Train data shape:",X_train.shape, "Train Label shape", y_train.shape)
print("Test data shape:",X_test.shape, "Test Label shape", y_test.shape)

# **Build a Feedforward Neural Network model**

**Input Layer:** Input(shape=(X_train.shape[1],)) creates the input layer with one neuron for each feature in the training data.

**First Hidden Layer:** A dense (fully connected) layer with 16 neurons. Each neuron learns from all the inputs of the previous layer.

**ReLU activation:** Outputs the input value if it’s positive, or 0 if it’s negative.

**Fully Connected Network:** Every neuron in one layer connects to all neurons in the next, each with its own weight, enabling the model to learn complex feature relationships.

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
    ])

model.summary() displays a layer-by-layer summary of the neural network, including the number of parameters and how data flows through the model.

From the model's summary, we can see that the model parameters, it has a total of 257 parameters, all of which are trainable, meaning they will be updated during training. There are no non-trainable parameters, so every weight and bias in the network is being learned from the data.

In [ ]:
model.summary()

[Extra] If you are curious how our model reached 257 parameters, here's a breakdown:
| Connection                   | Weights Calculation | Biases | Total Params |
| ---------------------------- | ------------------- | ------ | ------------ |
| Input (6) → Dense(16, ReLU)  | (6 * 16 = 96)  | 16     | 112          |
| Dense(16) → Dense(8, ReLU)   | (16 * 8 = 128) | 8      | 136          |
| Dense(8) → Dense(1, Sigmoid) | (8 * 1 = 8)    | 1      | 9            |
| **Total**                    | —                   | —      | **257**      |



# **Compiling the Model**

Before training, we need to compile the model by specifying how it will learn:

Optimizer = 'adam': An efficient variant of gradient descent that adapts the learning rate during training.

Loss = 'binary_crossentropy': We chose this because our Titanic task has only two possible outputs, either **survived** or **not survived**. This loss function is designed for binary classification problems.

Accuracy, Precision, and Recall: are evaluation metrics that track how often the model’s predictions match the true labels, giving an easy-to-understand measure of performance.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       Precision(name='precision'),
                       Recall(name='recall')])

# **Train the Neural Network Model**

**Validation Split:** Part of the training data is set aside (e.g., 80% train, 20% val) to monitor learning.

**Epoch:** One full pass through training data. More epochs help learning, but too many → overfitting.

**Loss** measures how far predictions are from true labels. We want this value to be as small as possible, ideally approaching 0.

**Accuracy** is the percentage of correct predictions. Values range from 0 to 1 (or 0% to 100%), and higher is better.

Same for both **precision** and **recall.** Values range from 0 to 1 (or 0% to 100%), and higher is better.

In [ ]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

To avoid overfitting when training with many epochs, we use **early stopping**, which automatically stops training once the validation performance stops improving.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=50,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=[early_stop])

Rather than feeding the entire dataset through the network in a single step, the training data is divided into smaller subsets called **batches**. The model processes one batch at a time and updates its weights after each batch

In [ ]:
history2 = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

**Epochs**

One epoch = full pass through training data (all batches).

Too few: underfitting (model hasn’t learned enough).

Too many: overfitting + wasted compute.

Recommendation: start with moderate epochs (e.g., 30–100) and use early stopping.

**Batch Size**

Definition: number of samples before a weight update.

Small (16–32): more updates, better generalization, slower per epoch.

Large (128–256): fewer updates, faster per epoch, higher memory use, risk of weaker generalization.

Common defaults: 32 or 64.

# **Visualize the training progress.**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot accuracy, precision, and recall from training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')

plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Val Precision')

plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Val Recall')

plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Training Progress (Accuracy, Precision, Recall)')
plt.legend()
plt.show()


# **Evaluate on the test set.**

In [ ]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)